# Download country lyrics

In [1]:
from urllib.request import Request, urlopen, quote # Python 3
from bs4 import BeautifulSoup
import requests
import json
import genius as genius_api
genius = genius_api.Genius()

### Get a list of each year's top 50 country artists from Billboard.com

In [2]:
def getArtistsFromList(URL):    
    # URL is a Billboard top list,
    # e.g. https://www.billboard.com/charts/year-end/2006/top-country-artists
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser")    
    chart_items = html.find_all("div", class_="ye-chart-item__title")
    return [item.get_text().strip() for item in chart_items]            

In [3]:
# Get the top 50 country artists each year since 2006
top_artists_by_year = {}
for year in range(2006,2018):    
    url = "https://www.billboard.com/charts/year-end/{}/top-country-artists".format(year)
    top_artists_by_year[year] = getArtistsFromList(url)

In [4]:
top_artists_by_year = {}
top_artists_by_year[2006] = ['Rascal Flatts', 'Sam Hunt']
top_artists_by_year[2016] = ['Chris Stapleton', 'Sam Hunt']

### Use the Genius API to get each artist's lyrics
https://github.com/johnwmillr/GeniusLyrics

In [19]:
# Use the Genius API to get each artist's lyrics
found_artists = {}
artist_lyrics = {}
for year in reversed(list(top_artists_by_year.keys())):
    print("\n******* {} *******".format(year))    
    artist_lyrics[year] = {'artists': []}
    for rank, name in enumerate(top_artists_by_year[year]):
        print("\n--------")
        artist_lyrics[year]['artists'].append({})
        if name not in found_artists:
            artist = genius.search_artist(name, max_songs=1)            
            artist_lyrics[year]['artists'][-1]['artist'] = artist.name
            artist_lyrics[year]['artists'][-1]['rank'] = rank+1                
            artist_lyrics[year]['artists'][-1]['songs'] = []
            for song in artist.songs:
                artist_lyrics[year]['artists'][-1]['songs'].append({})
                artist_lyrics[year]['artists'][-1]['songs'][-1]['title'] = song.title
                artist_lyrics[year]['artists'][-1]['songs'][-1]['album'] = song.album
                artist_lyrics[year]['artists'][-1]['songs'][-1]['year'] = song.year
                artist_lyrics[year]['artists'][-1]['songs'][-1]['lyrics'] = song.lyrics                
                artist_lyrics[year]['artists'][-1]['songs'][-1]['art'] = song.song_art_image_url            
            found_artists[name] = (name, year, len(artist_lyrics[year]['artists'])-1)
        else:
            # Store reference to artist location in dict, if artist previously found
            artist_lyrics[year]['artists'][-1] = found_artists[name]
            
    # Write the JSON object to disk as a backup before each new year
    with open('lyrics_{}.json'.format(year), 'w') as outfile:
        json.dump(artist_lyrics, outfile)
        
# Final write of the JSON object
with open('artist_lyrics.json', 'w') as outfile:
    json.dump(artist_lyrics, outfile)


******* 2016 *******

--------
Searching for Chris Stapleton...

Can't add song by Blake Shelton, artist must be Chris Stapleton.
Can't add song by Julie Roberts, artist must be Chris Stapleton.
Can't add song by Wynonna & The Big Noise, artist must be Chris Stapleton.
Can't add song by The SteelDrivers, artist must be Chris Stapleton.
Song 1: "A Simple Song"

Reached user-specified song limit (1).
Found 1 songs.

Done.

--------
Searching for Sam Hunt...

Can't add song by Kirk Knight & Nyck Caution, artist must be Sam Hunt.
Song 1: "Body Like a Back Road"

Reached user-specified song limit (1).
Found 1 songs.

Done.

******* 2006 *******

--------
Searching for Rascal Flatts...

Song 1: "Aftermath"

Reached user-specified song limit (1).
Found 1 songs.

Done.

--------


### Load and view the lyrics JSON object

In [22]:
from pprint import pprint
data = json.load(open('artist_lyrics.json'))
pprint(data)

{'2006': {'artists': [{'artist': 'Rascal Flatts',
                       'rank': 1,
                       'songs': [{'album': 'Rewind',
                                  'art': 'https://images.genius.com/b1e01574711602f1d3d4c8ffae498f5c.640x640x1.jpg',
                                  'lyrics': 'The way you turned your head\n'
                                            'And all the words you said\n'
                                            'The way they left your lips they '
                                            'left me\n'
                                            'Frozen in my tracks\n'
                                            "Now you can't take them back\n"
                                            "No it's too late for that, now I "
                                            'feel\n'
                                            '\n'
                                            'The aftermath of\n'
                                            'A fragile, cracked lo